In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
test_path = "../input/ranzcr-clip-catheter-line-classification/test/"
sample_df = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/sample_submission.csv")
model_path = "../input/model-for-ranzcr/model_b4.h5"
test_path = test_path + sample_df["StudyInstanceUID"] + ".jpg"
img_size = [244,240,260,300,380,456,528,600]
print(test_path[0])

In [ ]:
def decoder(size=(300,300)):
    def decode(path):
        file = tf.io.read_file(path)
        img = tf.io.decode_jpeg(file,channels=3)
        img = tf.cast(img,tf.float32) / 255
        img = tf.image.resize(img,size)
        return img
    return decode


def build_dataset(target,decode_fn = None,bsize=16):
    if decode_fn is None:
        decode_fn = decoder()
    auto = tf.data.experimental.AUTOTUNE
    dset = tf.data.Dataset.from_tensor_slices(target)
    dset = dset.map(decode_fn, num_parallel_calls=auto)
    dset = dset.batch(bsize).prefetch(auto)
    return dset

In [ ]:
test_decode = decoder(size=(img_size[4],img_size[4]))
test = build_dataset(test_path,decode_fn=test_decode,bsize=16)

In [ ]:
model = tf.keras.models.load_model(model_path)
model.summary()

In [ ]:
labels = sample_df.columns[1:]
sample_df[labels] = model.predict(test,verbose=1)
sample_df.to_csv("submission.csv",index=False)
sample_df.head()